In [ ]:
from copy import deepcopy

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2023, 1)
scenarios = ['ssp245', 'ssp245-constant-2005-aer']
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_2_0_obj = pooch.retrieve(
    url = 'https://zenodo.org/record/8399112/files/calibrated_constrained_parameters.csv',
    known_hash = 'md5:de3b83432b9d071efdd1427ad31e9076',
)

In [ ]:
df_configs = pd.read_csv(fair_params_1_2_0_obj, index_col=0)
configs = df_configs.index   # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
species, properties = read_properties(filename='../data/species_configs_properties_calibration1.2.0.csv')
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
da_emissions = xr.load_dataarray('../data/ssp245_emissions_1750-2022.nc')

In [ ]:
df_slcf = pd.read_csv('../data/slcf_emissions_1750-2022.csv', index_col=0)
df_slcf

In [ ]:
da = da_emissions.loc[dict(config="unspecified", scenario="ssp245")]
fe = da.expand_dims(dim=["scenario", "config"], axis=(1, 2))
for scenario in scenarios:
    for config in configs:
        f.emissions.loc[dict(scenario=scenario, config=config)] = fe.drop(("scenario", "config")).squeeze()
        f.emissions.loc[dict(scenario=scenario, config=config, specie='Sulfur')] = df_slcf['SO2']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='BC')] = df_slcf['BC']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='OC')] = df_slcf['OC']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='NOx')] = df_slcf['NOx']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='NH3')] = df_slcf['NH3']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='CO')] = df_slcf['CO']
        f.emissions.loc[dict(scenario=scenario, config=config, specie='VOC')] = df_slcf['NMVOC']

In [ ]:
f.emissions

In [ ]:
f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='Sulfur', timepoints=np.arange(2005.5, 2023))] = f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='Sulfur', timepoints=np.arange(1995.5, 2005))].mean().data[()]

In [ ]:
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245', specie='Sulfur', timepoints=np.arange(1990.5, 2023))
])
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245-constant-2005-aer', specie='Sulfur', timepoints=np.arange(1990.5, 2023))
])

In [ ]:
f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='BC', timepoints=np.arange(2005.5, 2023))] = f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='BC', timepoints=np.arange(1995.5, 2005))].mean().data[()]

In [ ]:
f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='OC', timepoints=np.arange(2005.5, 2023))] = f.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='OC', timepoints=np.arange(1995.5, 2005))].mean().data[()]

In [ ]:
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245', specie='BC', timepoints=np.arange(1990.5, 2023))
])
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245-constant-2005-aer', specie='BC', timepoints=np.arange(1990.5, 2023))
])

In [ ]:
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245', specie='OC', timepoints=np.arange(1990.5, 2023))
])
pl.plot(f.emissions.loc[
    dict(config=config, scenario='ssp245-constant-2005-aer', specie='OC', timepoints=np.arange(1990.5, 2023))
])

In [ ]:
solar_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-add-hfc/main/data/solar_erf_timebounds.csv',
    known_hash = 'md5:98f6f4c5309d848fea89803683441acf',
)

In [ ]:
volcanic_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-calibrate/main/data/forcing/volcanic_ERF_1750-2101_timebounds.csv',
    known_hash = 'md5:c0801f80f70195eb9567dbd70359219d',
)

In [ ]:
df_solar = pd.read_csv(solar_obj, index_col="year")
df_volcanic = pd.read_csv(volcanic_obj)

In [ ]:
solar_forcing = np.zeros(274)
volcanic_forcing = np.zeros(274)
volcanic_forcing = df_volcanic.erf.values[:274]
solar_forcing = df_solar["erf"].loc[1750:2023].values

trend_shape = np.ones(274)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs["fscale_Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs["fscale_solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["fscale_solar_trend"].values.squeeze(),
    specie="Solar",
)

In [ ]:
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "clim_c1":"clim_c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["clim_epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["clim_gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["clim_sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["clim_sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["clim_F_4xCO2"])

In [ ]:
f.fill_species_configs(filename='../data/species_configs_properties_calibration1.2.0.csv')

In [ ]:
# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["cc_r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs["cc_rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs["cc_rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs["cc_rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["aci_beta"].values.squeeze())
fill(f.species_configs["aci_shape"], df_configs["aci_shape_so2"].values.squeeze(), specie="Sulfur")
fill(f.species_configs["aci_shape"], df_configs["aci_shape_bc"].values.squeeze(), specie="BC")
fill(f.species_configs["aci_shape"], df_configs["aci_shape_oc"].values.squeeze(), specie="OC")

# aerosol direct
for specie in [
    "BC", 
    "CH4", 
    "N2O",
    "NH3", 
    "NOx",
    "OC", 
    "Sulfur", 
    "VOC",
    "Equivalent effective stratospheric chlorine"
]:
    fill(f.species_configs["erfari_radiative_efficiency"], df_configs[f"ari_{specie}"], specie=specie)

# forcing scaling
for specie in [
    "CO2", 
    "CH4", 
    "N2O", 
    "Stratospheric water vapour",
    "Contrails", 
    "Light absorbing particles on snow and ice", 
    "Land use"
]:
    fill(f.species_configs["forcing_scale"], df_configs[f"fscale_{specie}"].values.squeeze(), specie=specie)
# the halogenated gases all take the same scale factor
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(f.species_configs["forcing_scale"], df_configs["fscale_minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(f.species_configs["ozone_radiative_efficiency"], df_configs[f"o3_{specie}"], specie=specie)

# initial value of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"], 
    df_configs["cc_co2_concentration_1750"].values.squeeze(), 
    specie="CO2"
)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.temperature.loc[dict(layer=0)].median(axis=2))

In [ ]:
g = FAIR(ch4_method="Thornhill2021")
g.define_time(1750, 2023, 1)
g.define_scenarios(scenarios)
g.define_configs(configs)

In [ ]:
properties_noaer = deepcopy(properties)

In [ ]:
properties_noaer

In [ ]:
properties_noaer['Aerosol-cloud interactions']['input_mode'] = 'forcing'
properties_noaer['Aerosol-radiation interactions']['input_mode'] = 'forcing'

In [ ]:
g.define_species(species, properties_noaer)

In [ ]:
g.allocate()

In [ ]:
da = da_emissions.loc[dict(config="unspecified", scenario="ssp245")]
fe = da.expand_dims(dim=["scenario", "config"], axis=(1, 2))
for scenario in scenarios:
    for config in configs:
        g.emissions.loc[dict(scenario=scenario, config=config)] = fe.drop(("scenario", "config")).squeeze()
        g.emissions.loc[dict(scenario=scenario, config=config, specie='Sulfur')] = df_slcf['SO2']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='BC')] = df_slcf['BC']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='OC')] = df_slcf['OC']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='NOx')] = df_slcf['NOx']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='NH3')] = df_slcf['NH3']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='CO')] = df_slcf['CO']
        g.emissions.loc[dict(scenario=scenario, config=config, specie='VOC')] = df_slcf['NMVOC']

In [ ]:
g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='Sulfur', timepoints=np.arange(2005.5, 2023))] = g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='Sulfur', timepoints=np.arange(1995.5, 2005))].mean().data[()]
g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='BC', timepoints=np.arange(2005.5, 2023))] = g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='BC', timepoints=np.arange(1995.5, 2005))].mean().data[()]
g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='OC', timepoints=np.arange(2005.5, 2023))] = g.emissions.loc[dict(scenario='ssp245-constant-2005-aer', specie='OC', timepoints=np.arange(1995.5, 2005))].mean().data[()]

In [ ]:
fill(
    g.forcing,
    volcanic_forcing[:, None, None] * df_configs["fscale_Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    g.forcing,
    solar_forcing[:, None, None] * df_configs["fscale_solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs["fscale_solar_trend"].values.squeeze(),
    specie="Solar",
)

In [ ]:
fill(g.forcing, 0, specie='Aerosol-radiation interactions')
fill(g.forcing, 0, specie='Aerosol-cloud interactions')

In [ ]:
g.fill_species_configs(filename='../data/species_configs_properties_calibration1.2.0.csv')

In [ ]:
fill(g.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "clim_c1":"clim_c3"].values)
fill(
    g.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)
fill(g.climate_configs["deep_ocean_efficacy"], df_configs["clim_epsilon"].values.squeeze())
fill(g.climate_configs["gamma_autocorrelation"], df_configs["clim_gamma"].values.squeeze())
fill(g.climate_configs["sigma_eta"], df_configs["clim_sigma_eta"].values.squeeze())
fill(g.climate_configs["sigma_xi"], df_configs["clim_sigma_xi"].values.squeeze())
fill(g.climate_configs["seed"], df_configs["seed"])
fill(g.climate_configs["stochastic_run"], False)
fill(g.climate_configs["use_seed"], True)
fill(g.climate_configs["forcing_4co2"], df_configs["clim_F_4xCO2"])

In [ ]:
# carbon cycle
fill(g.species_configs["iirf_0"], df_configs["cc_r0"].values.squeeze(), specie="CO2")
fill(g.species_configs["iirf_airborne"], df_configs["cc_rA"].values.squeeze(), specie="CO2")
fill(g.species_configs["iirf_uptake"], df_configs["cc_rU"].values.squeeze(), specie="CO2")
fill(g.species_configs["iirf_temperature"], df_configs["cc_rT"].values.squeeze(), specie="CO2")

# aerosol indirect
fill(g.species_configs["aci_scale"], df_configs["aci_beta"].values.squeeze())
fill(g.species_configs["aci_shape"], df_configs["aci_shape_so2"].values.squeeze(), specie="Sulfur")
fill(g.species_configs["aci_shape"], df_configs["aci_shape_bc"].values.squeeze(), specie="BC")
fill(g.species_configs["aci_shape"], df_configs["aci_shape_oc"].values.squeeze(), specie="OC")

# aerosol direct
for specie in [
    "BC", 
    "CH4", 
    "N2O",
    "NH3", 
    "NOx",
    "OC", 
    "Sulfur", 
    "VOC",
    "Equivalent effective stratospheric chlorine"
]:
    fill(g.species_configs["erfari_radiative_efficiency"], df_configs[f"ari_{specie}"], specie=specie)

# forcing scaling
for specie in [
    "CO2", 
    "CH4", 
    "N2O", 
    "Stratospheric water vapour",
    "Contrails", 
    "Light absorbing particles on snow and ice", 
    "Land use"
]:
    fill(g.species_configs["forcing_scale"], df_configs[f"fscale_{specie}"].values.squeeze(), specie=specie)
# the halogenated gases all take the same scale factor
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(g.species_configs["forcing_scale"], df_configs["fscale_minorGHG"].values.squeeze(), specie=specie)

# ozone
for specie in ["CH4", "N2O", "Equivalent effective stratospheric chlorine", "CO", "VOC", "NOx"]:
    fill(g.species_configs["ozone_radiative_efficiency"], df_configs[f"o3_{specie}"], specie=specie)

# initial value of CO2 concentration (but not baseline for forcing calculations)
fill(
    g.species_configs["baseline_concentration"], 
    df_configs["cc_co2_concentration_1750"].values.squeeze(), 
    specie="CO2"
)

In [ ]:
initialise(g.concentration, g.species_configs["baseline_concentration"])
initialise(g.forcing, 0)
initialise(g.temperature, 0)
initialise(g.cumulative_emissions, 0)
initialise(g.airborne_emissions, 0)

In [ ]:
g.run()

In [ ]:
pl.plot(g.temperature.loc[dict(layer=0)].median(axis=2))

In [ ]:
pl.plot((f.temperature.loc[dict(layer=0)] - g.temperature.loc[dict(layer=0)]).median(axis=2))

In [ ]:
pl.plot(f.forcing_sum.median(axis=2))
pl.ylim(0, 3.1)

In [ ]:
pl.plot(g.forcing_sum.median(axis=2))
pl.ylim(1.5, 4.6)

In [ ]:
pl.plot((f.forcing_sum-g.forcing_sum).median(axis=2))

In [ ]:
f.forcing_sum.median(axis=2)

In [ ]:
pl.plot((f.forcing_sum).median(axis=2))

In [ ]:
pl.plot((g.forcing_sum).median(axis=2))

In [ ]:
pl.plot(np.percentile(f.temperature.loc[dict(layer=0)] - g.temperature.loc[dict(layer=0)], 5, axis=2))
pl.plot((f.temperature.loc[dict(layer=0)] - g.temperature.loc[dict(layer=0)]).median(axis=2))
pl.plot(np.percentile(f.temperature.loc[dict(layer=0)] - g.temperature.loc[dict(layer=0)], 95, axis=2))

In [ ]:
pl.plot(np.percentile(f.temperature.loc[dict(layer=0)], 5, axis=2))
pl.plot((f.temperature.loc[dict(layer=0)]).median(axis=2))
pl.plot(np.percentile(f.temperature.loc[dict(layer=0)], 95, axis=2))